# ASI Hunting - SecurityEvent Template

We would like to guide you though your threat hunting efforts:
    1. Prepare the shell;
    2. Enter values;
    3. Execute queries;
    4. Analyze data.


## 0. Retrieve Log Analytics Information

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML

from utils import config_reader as mod
workspaceId = mod.config_reader.read_config_values("../config.json")[3]
tenantId = mod.config_reader.read_config_values("../config.json")[0]

######################################################################
your_workspace_Name = mod.config_reader.read_config_values("../config.json")[4]
print("Your Log Analytic Workspace: " + your_workspace_Name)
######################################################################

## 1. Install KQL Magic for Log Analytics

In [ ]:
# run once unless upgrade
!pip install Kqlmagic --no-cache-dir --upgrade

In [ ]:
%reload_ext Kqlmagic

If you need any help for kql magic     --> %kql --help "help" -popup_window
If you need any help for Log Analytics --> %kql --help "LogAnalytics"

## 2. Log into Log Analytics

In [ ]:
%kql loganalytics://code;workspace=workspaceId;tenant=tenantId;alias="asidb" -!se

In [ ]:
dbSchema = %kql --schema "asidb@loganalytics"
tables = list(dbSchema.keys())
selectedTable = widgets.Dropdown(options=tables, value=tables[0],description='Data Table:')
display(selectedTable)

## 3. Data Analysis

In [ ]:
# Load queries from GitHub
import requests
import json
url = "https://raw.githubusercontent.com/ASIJupyter/ASINotebooks/master/queries/Azure%20Network%20Analytics.json"
response = requests.get(url)
resJson = json.loads(response.text)
if response != None:
    try:
        print(resJson["query"])
    except:
        print(type("type: " + response.text))

In [ ]:
# run the query
%kql -query=resJson["query"]
if len(_kql_raw_result_) > 0:
    df = _kql_raw_result_.to_dataframe()
else:
    df = None
    print('no data')

In [ ]:
import pandas
newdf = df.assign(ipList=df.ipArray.apply(lambda x: ', '.join(map(lambda y: str(y).strip('[]').strip('\''), x))))
newdf = newdf.drop('ipArray', axis=1)

In [ ]:
newdf

In [ ]:
newdf.to_csv('ipdata')